# 基本操作

In [1]:
import tensorflow as tf
import numpy as np
##先引入对应的包

## Graph、Node、Tensor

tensorflow的计算是通过一个计算图(computation graph，有时也称有向图,directed graph)模型来做的，使用tensorflow完成一个目标就是需要搭建一个对应的图模型，一个图可以分解为**Node节点（一个节点表示一个操作，operation）**、**在图中流动的tensor**、**节点之间的连接边（edge）**。Node可看做是一个计算节点，接受0个或多个tensor作为输入，输出1个或多个tensor。tensor（张量），可以类比于多维数组，0阶张量表示为一个常数（常量、纯量）；1阶张量则可表示为1维数组（向量）；2阶张量表示为2维数组（矩阵）。

In [2]:
#创建了两个常量Node，接受0个输入，1个输出
cons1 = tf.constant(3.);
cons2 = tf.constant(2.);
#创建了第三个节点Node，与前两个节点连接，此时也是一个常量节点
add_node = tf.add(cons1,cons2)

print(add_node)

Tensor("Add:0", shape=(), dtype=float32)


## Session

TensorFlow的计算任务需要在对应的上下文Session中进行

In [3]:
with tf.Session() as sess:
    print(sess.run([cons1, cons2 , add_node]))

[3.0, 2.0, 5.0]


## placeholder、variable

In [4]:
#placeholder可以参数化输入，表示介绍一个3*2(2*3)大小的2阶张量（矩阵）
a = tf.placeholder(dtype=tf.float32,shape=(3,2))
b = tf.placeholder(dtype=tf.float32,shape=(2,3))
#c为两矩阵的乘积
c=tf.matmul(a,b)

In [5]:
#模拟两个输入
input1 = np.random.rand(3,2)
input2 = np.random.rand(2,3)
#进行计算，注意需要传入feed_dict参数，作为代替c节点中的参数
with tf.Session() as sess:
    print(sess.run( c, feed_dict={a:input1,b:input2}))

[[ 0.22469544  0.87722647  0.79920864]
 [ 0.12417199  0.48619077  0.45701045]
 [ 0.09483597  0.36098951  0.23689401]]


In [6]:
#placeholder表示外界的输入，而Variable则表示可训练的模型自身的参数，用来保存模型的状态
#可以被后续的操作更新
W = tf.Variable([.1],dtype=tf.float32)
b = tf.Variable([-.1],dtype=tf.float32)
x = tf.placeholder(tf.float32)
custom_lm = W * x + b
#由于variable在创建时不会自动的初始化，于是需要显示的调用初始化语句
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) #先初始化每个参数，init这个op可以初始化所有参数
    print(sess.run(custom_lm,{x:[1,2,3,4]}))

[ 0.          0.1         0.20000002  0.30000001]


## loss function and training

In [8]:
input_x=np.array([1.1,2.03,2.95,4.08,5.05])
input_y=np.array([0.95,1.98,3.11,4.00,4.99])
#对于一个模型需要构建一个其的loss function
y = tf.placeholder(tf.float32)
loss = tf.reduce_sum(tf.square(custom_lm-y))#二次损失函数
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(loss , {x : input_x , y : input_y}))

47.557


In [9]:
#之后可以使用训练tf.train API进行模型的训练
#使用梯度下降进行优化
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
with tf.Session() as sess:
    sess.run(init)
    #1000次迭代训练
    for _ in range(10):
        sess.run(train,{x:input_x,y:input_y})
    #评估最后的结果
    W_,b_,loss_ = sess.run( [ W , b , loss ],{x:input_x,y:input_y})
    print(W_,b_,loss_)

[ 0.96384549] [ 0.10305085] 0.080245


## TersorBoard & Visualization

可视化可以参考[TensorBoard README](https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/tensorboard/README.md)以及[TensorBoard](https://www.tensorflow.org/get_started/summaries_and_tensorboard)
[TensorFlowSubmit](https://www.youtube.com/watch?v=eBbEDRsCmv4&index=5&list=PLOU2XLYxmsIKGc_NBoIhTn2Qhraji53cv)

可视化主要是通过启动tensorboard，来可视化计算的值，而将tensorflow的计算过程传递给tensorboard实际上需要summary操作，并将计算的结果存入磁盘里，tensorboard则通过读取这些信息来进行可视化

一个简单的步骤为：
1. 对需要生产summary的节点添加对应的summary操作，所有的summary操作参见[summary operation](https://www.tensorflow.org/api_guides/python/summary)
2. 执行summary操作。因为summary操作可能不被其他节点依赖，所以需要显式执行，可以使用[tf.summary.merge_all](https://www.tensorflow.org/api_docs/python/tf/summary/merge_all)将所有的summary生成一个节点进行执行
3. 执行完summary操作后会生成一个序列化对象，然后持久化保存此对象.[tf.summaryFileWriter](https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter)

In [10]:
#记录loss,W,b
tf.summary.scalar('loss',loss)
tf.summary.tensor_summary('weight',W)
tf.summary.tensor_summary('bias',b)

merged = tf.summary.merge_all()


In [11]:
with tf.Session() as sess:
    sess.run(init)
    #初始化FileWriter
    fileWriter = tf.summary.FileWriter( 'logs/basicOP',sess.graph)
    #1000次迭代训练
    for i in range(10):
        _ , summaried= sess.run([train,merged],{x:input_x,y:input_y})
        fileWriter.add_summary(summaried,i)
    #评估最后的结果
    W_,b_,loss_ = sess.run( [ W , b , loss ],{x:input_x,y:input_y})
    print(W_,b_,loss_)

NameError: name 'tf' is not defined

运行完后即可以启动tensorboard，设置好对应的logdir后即可查看运行的情况
`tensorboard --logdir=path/to/logs`
启动后进入localhost:6006